In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
#from folium.element import IFrame
import shapely
from shapely.geometry import Point, Polygon
import unicodedata
#import pysal as ps
from IPython.display import IFrame
import jenkspy
import seaborn as sns
import matplotlib.pyplot as plt
import branca.colormap as cm

def my_color_function(feature):
    cluster=feature["properties"]["em_grid_risk"]
    fill_color=color='#fdd4c2'

    if(cluster=="Very Low"):
        return "#d9d9ea"
    elif(cluster=="Low"):
        return "#9d99c7"
    elif(cluster=="Medium"):
        return "#6950a2"
    elif(cluster=="High"):
        return "#e32f27"
    elif(cluster=="Very High"):
        return "#b11117"
    else:
        return fill_color
        
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    listofcollor=["#fee186","#fdaa48","#fc5a2d","#d30f20"]

    marker_cluster = folium.MarkerCluster()

    for i, row in gdf.iterrows():
        cluster=row["risk_cluster"]
        numberofviolations=str(row["number_of_violations"])
        label = 'Number of Violations : '+numberofviolations
        color="#c5daee"
        if(cluster=="Very Low"):
            color="#6aadd5"
        elif(cluster=="Low"):
            color="#2070b4"
            #folium.PolygonMarker([row.geometry.y, row.geometry.x], popup=label, color=color,fill_color=color, fill_opacity=3, weight=4,radius=2).add_to(mapobj)
             #           radius=12, popup=(vis1, 'vis1.json'))
        elif(cluster=="High"):
            color="#2070b4"
        elif(cluster=="Very High"):
            color="#b80a4e"
        circlemarker1=folium.CircleMarker([row.geometry.y, row.geometry.x], popup=label, color=color,fill_color=color, fill_opacity=3, weight=4,radius=2).add_to(mapobj)
   


    minval=gdf['number_of_violations'].min()
    maxval=gdf['number_of_violations'].max()

    colormap=cm.linear.PuRd.scale(minval, maxval)
    colormap.caption = 'Number of Violations'
    mapobj.add_child(colormap)


    return mapobj


violationdf=pd.read_csv('LivingstonLocationCount.csv',keep_default_na=False)
violoation_geo = gpd.GeoSeries(violationdf.apply(lambda z: Point(z['longitude'], z['latitude']), 1),crs={'init': 'epsg:4326'})
violationdf = gpd.GeoDataFrame(violationdf.drop(['latitude', 'longitude'], 1), geometry=violoation_geo)
#BUSHC 40.5212, -74.4623
#COOK 40.4793 7-4.4332 
#livingstong 40.5240 -74.4355
#4COLLEGE AVE 40.4982° N, 74.4468° W
violation_map = folium.Map([40.5240, -74.4355], zoom_start = 15)
#violation_map = folium.Map([40.5008, -74.4474], zoom_start = 13)

violation_map = add_point_clusters(violation_map, violationdf, ['number_of_violations'])

griddf=pd.read_csv('/GridCount.csv',keep_default_na=False)
griddf=griddf.loc[(griddf.campus=="Livingston" )]#"College Ave")]

griddf_geo = gpd.GeoSeries(griddf.apply(lambda z:  
                                                 Polygon([(z['grid8_pointx1'], z['grid8_pointy1']), (z['grid8_pointx2'], z['grid8_pointy2']), (z['grid8_pointx3'], z['grid8_pointy3']),(z['grid8_pointx4'],z['grid8_pointy4'])]),axis=1),crs={'init': 'epsg:4326'})
griddf = gpd.GeoDataFrame(griddf.drop(['grid8_pointx1','grid8_pointy1','grid8_pointx2','grid8_pointy2','grid8_pointx3','grid8_pointy3','grid8_pointx4','grid8_pointy4'], axis=1), geometry=griddf_geo)
griddf["popup"]=griddf["em_grid_risk"]
gridgeojson = folium.GeoJson(
    griddf,
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color':  my_color_function(feature),
        'weight':1,
        'fillOpacity':0.9,
        'lineOpacity' : 4,
    
    
    }
).add_to(violation_map)

folium.LayerControl().add_to(violation_map)
violation_map.save('LivingstonLocationMap.html')

violation_map
